# WEB SCRAPING

#  A - Preparación

In [ ]:
pip install beautifulsoup4

In [ ]:
!pip install db-sqlite3

In [ ]:
from bs4 import BeautifulSoup
import requests
from csv import writer
from datetime import datetime
import sqlite3
import itertools
import json

# B - Creación de funciones

## 1 - Auxiliar

In [ ]:
def get_fields(url):
  html_text = requests.get(url).text
  soup = BeautifulSoup(html_text, 'lxml')
  
  table = soup.find('div', class_ = "table_container current")
  list_fields = []
  passes_blocked = 0
  
  #Get statistics
  for tr in table.find_all('tr')[2:]:
    has_class = tr.has_attr('class')
    if not has_class or (has_class and tr['class'][0] != 'thead' and tr['class'][0] != 'partial_table' and tr['class'][0] != 'spacer'):
      categoria = tr.find('th').text
      #There are two categories called Passes Blocked, the first in attack the second in defense
      if categoria == 'Passes Blocked':
        if passes_blocked == 1:
          categoria = 'Passes Blocked in Defense'
        else:
          passes_blocked = passes_blocked + 1
      list_fields.append(categoria)
      if(hasPercentile):
        list_fields.append(categoria+'_percentile')

  #return data    
  return list_fields

## 2 - Inserción en la base de datos

In [ ]:
def insert_table_contrato(contratos):
  #Insert contratos into table Contrato
  cursor = db.cursor()

  cursor.executemany("INSERT INTO Contrato (id_equipo, id_jugador, temporada, matchs_played) VALUES (?,?,?,?)", contratos)

  db.commit()

In [ ]:
def create_INSERT_statement(table_name, columns):
  place_holder = "(?, ?, ?, "
  statement = "INSERT INTO "+ table_name +" (id_jugador, id_campeonato, temporada, "
  first = True

  for column in columns:
    column = format_name(column)
    if not first:
      statement = statement + ', ' + column
      place_holder = place_holder + ', ?'
    else:
      first = False
      statement = statement + column
      place_holder = place_holder + '?'

  statement = statement + ') VALUES' + place_holder + ')'

  return statement

In [ ]:
def insert_table_estadistica_portero(estadistica):
  table_name = "Estadisticas_portero"
  columns = fields_porteros
  
  statement = create_INSERT_statement(table_name, columns)
  cursor = db.cursor()

  cursor.executemany(statement, estadistica)
  
  db.commit()

In [ ]:
def insert_table_estadistica_jugador(estadistica):
  table_name = "Estadisticas_jugador"
  columns = fields_jugadores
  
  statement = create_INSERT_statement(table_name, columns)

  cursor = db.cursor()

  cursor.executemany(statement, estadistica)
  
  db.commit()


In [ ]:
def insert_table_jugador(jugadores):
  #Insert jugador into table Jugador
  cursor = db.cursor()

  cursor.executemany("INSERT INTO Jugador (jugador_id ,nombre, nacionalidad, peso, altura, fecha_nacimiento, GK, DF, MF, FW, CB, FB, DM, CM, AM, WM) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", jugadores)

  db.commit()

In [ ]:
def insert_table_resultado(resultados):
  #Insert resultados into table Resultado
  cursor = db.cursor()

  cursor.executemany("INSERT INTO Resultado (id_equipo, id_campeonato, temporada, posicion, puntos) VALUES (?,?,?,?,?)", resultados)
 
  db.commit()

In [ ]:
def insert_table_equipo(equipos):
  #Insert equipos into table Equipos
  cursor = db.cursor()

  cursor.executemany("INSERT INTO Equipo (equipo_id, nombre, pais) VALUES (?,?,?)", equipos)

  db.commit()

## 3 - Extracción de la página web

In [ ]:
def get_country_from_league(soup):
  #recibe el soup de una liga y devuelve el pais en el que se juega
  country = soup.find('div',id='meta').find_all('div')[1].find_all('p')[0].find('a').text
  return country

In [ ]:
def get_contrato_from_player(soup, jugador_id, diccionario_equipos):
  #recibe el soup de dom_lg y el id del jugador y devuelve una lista con todos los contratos tambien inserta nuevos equipos en la tabla equipos
  contratos = []
  equipos = []
  table = soup.find('table', class_='stats_table')

  for row in table.find('tbody').find_all('tr', id='stats'):
    contrato = []
    season = row.find('th', class_='left').text
    stats = row.find_all('td')
    
    team_name = stats[1].text
    html_team = stats[1].find('a', href=True)['href']
    
    equipo_id = html_team.split('/')[3]
    
    if equipo_id not in diccionario_equipos :
      html_league = stats[2].find('a', href=True)['href']
      split = html_league.split('/')[4].split('-')
      split = split[:len(split)-1]
      country = " ".join(word for word in split)
      equipos.append([equipo_id,team_name,country])
      diccionario_equipos[equipo_id] = team_name

    matchs_played = stats[5].text
    contrato.append(equipo_id)
    contrato.append(jugador_id)
    contrato.append(season)
    contrato.append(int(matchs_played))
    
    contratos.append(contrato)   
  if equipos: #check not empty 
    insert_table_equipo(equipos)
  return contratos, diccionario_equipos

In [ ]:
def get_estadisticas_from_player(soup, jugador_id, season, league, portero):
  #recibe el soup del scouting report de una temporada la temporada y la liga en string, devuelve la lista que se inserta en la tabla estadistica
  estadistica = []
  passes_blocked = 0
  
  if portero:
    diccionario_fields = dict(diccionario_fields_porteros)
  else:
    diccionario_fields = dict(diccionario_fields_jugadores)

  estadistica.append(jugador_id)

  campeonato_id = diccionario_ligas[league]
  estadistica.append(campeonato_id)

  estadistica.append(season)

  table = soup.find('div', class_ = "table_container current")
  
  for tr in table.find_all('tr')[2:]:
    has_class = tr.has_attr('class')
    if not has_class or (has_class and tr['class'][0] != 'thead' and tr['class'][0] != 'partial_table' and tr['class'][0] != 'spacer'):
        categoria = tr.find('th').text
        #There are two categories called Passes Blocked, the first in attack the second in defense
        if categoria == 'Passes Blocked':
          if passes_blocked == 1:
            categoria = 'Passes Blocked in Defense'
          else:
            passes_blocked = passes_blocked + 1
            
        valor = tr.find('td', class_ ='right').text
        diccionario_fields[categoria] = valor.replace('+','').replace('%','')
        if(hasPercentile):
          percentile = tr.find('div', align ='right').text
          diccionario_fields[categoria+'_percentile'] = percentile.replace('+','').replace('%','')
        #estadistica.append(valor.replace('+','').replace('%',''))
  return estadistica+list(diccionario_fields.values())

In [ ]:
def get_all_estadisticas_from_player(soup, jugador_id):
  #recibe el soup del scouting report, devuelva una lista con todas las estadisticas de todas las temporadas de un jugador
  all_estadisticas = []
  portero = False
  table = soup.find('div', class_ = "table_container current")

  if table is None:
    return None, None
    
  tr = table.find_all('tr')[2:][0]
  categoria = tr.find('th').text

  if(categoria!='Goals'):
    portero = True
  
  for a in soup.find('div', class_='filter').find_all('a', href=True):
    #Case: Last 365 Days Major League Soccer
    if 'Last' not in a.text:
      #Case: 2018 Major League Soccer
      if '-' in a.text:
        season = a.text[:9]
        league = a.text[10:]
      else:
        season = a.text[:4]
        league = a.text[5:]

      seasons_url = a['href']

      html_text = requests.get(web_page + seasons_url).text
      soup = BeautifulSoup(html_text, 'lxml')

      all_estadisticas.append(get_estadisticas_from_player(soup, jugador_id, season, league, portero))

  return all_estadisticas, portero

In [ ]:
def get_jugador_from_player(soup, jugador_id):
  #recibe el soup del scouting report, devuelva la lista que se inserta en la tabla jugador y su id

  table = soup.find('div', class_ = "table_container current")
  info = soup.find('div', itemtype='https://schema.org/Person')
  jugador = [0] * 16
  no_pos = False

  
  if table is None:
    return None
  

  #Get info
  #Case: there is no complete name
  if(info.find_all('p')[2].text[:8] == 'Position'):
    p = 2
  elif(info.find_all('p')[1].text[:8] == 'Position'):
    p = 1
  elif(info.find_all('p')[0].text[:8] == 'Position'):
    p = 0
  else:
    p = -1
  
  if not p == -1:
    #main position
    pos_row = info.find_all('p')[0+p].text.split(' ')
    if "-" in pos_row[1]:
      pos_main = pos_row[1].split("-")

      list_pos = diccionario_posiciones[pos_main[0]]
      jugador[list_pos] = 1


      if "(" not in pos_row:
        pos_main[1] = pos_main[1].split("\xa0")[0]

      list_pos = diccionario_posiciones[pos_main[1]]
      jugador[list_pos] = 1

    else:

      list_pos = diccionario_posiciones[pos_row[1][:2]]
      jugador[list_pos] = 1
      
    #specific position
    if len(pos_row) > 2:
      if "(" in pos_row[2]:
        pos_sec = pos_row[2].split(',')[0].split("(")
        pos_sec = pos_sec[1].split(")")[0]
        if "-" in pos_sec:
          pos_sec = pos_sec.split("-")

          list_pos = diccionario_posiciones[pos_sec[0]]
          jugador[list_pos] = 1

          list_pos = diccionario_posiciones[pos_sec[1]]
          jugador[list_pos] = 1
        else:
          list_pos = diccionario_posiciones[pos_sec]
          jugador[list_pos] = 1

  jugador[0] = jugador_id
  
  name = soup.find('h1', itemprop='name').find('span').text
  jugador[1] = name 

  weight = soup.find('span', itemprop='weight')
  if weight is not None:
    weight = weight.text.replace('kg','')
  jugador[3] = weight

  height = soup.find('span', itemprop='height')
  if height is not None:
    height = height.text.replace('cm','')
  jugador[4] = height

  #Case: there is no line for weight and height
  if height is None and weight is None:
    p = p - 1 

  date_row = info.find_all('p')[2+p].text.split(':')[1].split("  ")[2]
  date_text = date_row[:len(date_row)-1]
  date_object = datetime.strptime(date_text, "%B %d, %Y").strftime("%d/%m/%Y")
  jugador[5] = date_object
  
  country = info.find_all('p')[3+p].text.split(':')[1].split(' ')[0][1:]
  jugador[2] = country
 
  return jugador


In [ ]:
def get_all_player_from_team(soup, diccionario_jugadores, diccionario_equipos):
  #recibe el soup del equipo
  jugadores = []
  estadisticas_jugador = []
  estadisticas_portero = []
  contratos = []

  table = soup.find('div', id = 'all_stats_standard').find('div', class_ = "table_container")

  for row in table.find('tbody').find_all('th', scope = 'row'):
    href = row.find(href=True)
    #Case: Benevento 2017-2018 Christian Rutjens
    if href is not None:
      player_name = href.text
      url_jugador = href['href']
      split = url_jugador.split('/')
      url_jugador_stats = web_page + 'en/players/' + split[3] + '/scout/365_euro/' + split[4] + '-Scouting-Report'

      jugador_id = split[3]

      #solo si no se encuentra en el diccionario de todos los equipos, sino ya deberia tener sus estadisticas, jugador y contrato
      if jugador_id not in diccionario_jugadores:
        html_text = requests.get(url_jugador_stats).text
        soup = BeautifulSoup(html_text, 'lxml')

        print(player_name, end=" ")
        #number_jugador_id = jugador_id.siguiente()
        print(jugador_id)
        diccionario_jugadores[jugador_id] = player_name

        jugador = get_jugador_from_player(soup, jugador_id)
        estadistica, portero = get_all_estadisticas_from_player(soup, jugador_id)

        #Case: player has no estadistica
        if estadistica is not None and jugador is not None:
          jugadores.append(jugador)
          if portero:
            estadisticas_portero.extend(estadistica)
          else:
            estadisticas_jugador.extend(estadistica)

          url_jugador_leagues = web_page + 'en/players/' + split[3] + '/dom_lg/' + split[4] + '-Domestic-League-Stats'
          html_text = requests.get(url_jugador_leagues).text
          soup_leagues = BeautifulSoup(html_text, 'lxml')
          contrato, diccionario_equipos = get_contrato_from_player(soup_leagues, jugador_id, diccionario_equipos)

          if contrato is not None:
            contratos.extend(contrato)

        else:
          diccionario_jugadores.pop(jugador_id)
        
  insert_table_jugador(jugadores)
  insert_table_estadistica_portero(estadisticas_portero)  
  insert_table_estadistica_jugador(estadisticas_jugador)
  insert_table_contrato(contratos)

  return diccionario_jugadores

  #recibe un soup del equipo
  #sacar la url de todos los jugadores, ir al scouting report y a domestic league
  #en scouting report rellenar tabla jugadores y tabla estadisticas
  #en domestic league rellenar tabla contrato

In [ ]:
def get_all_team_from_ligue(soup, ligue_name, diccionario_equipos, diccionario_jugadores):
  #recibe el soup de la clasificacion de la liga en una temporada
  table = soup.find('table', class_='stats_table')
  list_url = []
  equipos = []
  resultado = []

  country = soup.find('div',id='meta').find_all('div')[1].find_all('p')[0].find('a').text
  season = soup.find('h1', itemprop='name').text.split(' ')[0][10:]

  for row in table.find('tbody').find_all('tr'):
    team = row.find_all('td', class_='left')[0].find(href=True)
    team_url = team['href']
    team_name = team.text

    html_text = requests.get(web_page + team_url).text
    soup = BeautifulSoup(html_text, 'lxml')
    print(team_name)
    print(season)
    diccionario_jugadores = get_all_player_from_team(soup, diccionario_jugadores, diccionario_equipos)
    
    equipo_id = team_url.split('/')[3]
    
    #solo si no esta ya en el diccionario de todos los equipos
    if equipo_id not in diccionario_equipos:
      equipos.append([equipo_id,team_name,country])
      diccionario_equipos[equipo_id] = team_name

    team_stats = row.find_all('td', class_='right')
    match_played = team_stats[0].text
    match_wins = team_stats[1].text
    match_draws = team_stats[2].text
    match_loses = team_stats[3].text
    team_points = team_stats[7].text
    position = row.find('th', scope='row').text

    resultado.append([equipo_id, diccionario_ligas[ligue_name],season,position,team_points])

  insert_table_equipo(equipos)
  insert_table_resultado(resultado)

  return diccionario_equipos, diccionario_jugadores

In [ ]:
def get_ligue_all_years(url_ligue , ligue_name, diccionario_equipos, diccionario_jugadores):
  max_iter = 4 #para que recoja hasta la temporada 2017-2018
  i = 0

  url_ligue = web_page + url_ligue
  html_text = requests.get(url_ligue).text
  soup = BeautifulSoup(html_text, 'lxml')

  #una vez que tengo soup, extraer la información de la clasificación de la liga y todos los equipos
  #para cada equipo, entrar y obtener todos los jugadores/estadísticas
  diccionario_equipos, diccionario_jugadores = get_all_team_from_ligue(soup, ligue_name, diccionario_equipos, diccionario_jugadores)

  while i<max_iter:
    url_ligue_prev = soup.find('div', class_= 'prevnext').a['href']
    url_ligue_prev = web_page + url_ligue_prev
    
    html_text = requests.get(url_ligue_prev).text
    soup = BeautifulSoup(html_text, 'lxml')

    diccionario_equipos, diccionario_jugadores = get_all_team_from_ligue(soup, ligue_name, diccionario_equipos, diccionario_jugadores)

    i = i + 1

  return diccionario_equipos, diccionario_jugadores


## 4 - Creación de la base de datos

In [ ]:
def create_database(db_name):
  data_base = sqlite3.connect(db_name)
  return data_base

In [ ]:
def create_and_fill_table_campeonato():
  cursor = db.cursor()

  cursor.execute("""CREATE TABLE Campeonato (
      nombre text,
      campeonato_id INTEGER PRIMARY KEY
    )""")

  cursor.executemany("INSERT INTO Campeonato VALUES (?,?)", diccionario_ligas.items())

  cursor.execute("SELECT * FROM Campeonato")
  db.commit

In [ ]:
def create_table_equipo():
  cursor = db.cursor()

  cursor.execute("""CREATE TABLE Equipo (
      equipo_id text,
      nombre text,
      pais text
    )""")

  db.commit

In [ ]:
def create_table_resultado():
  cursor = db.cursor()

  cursor.execute("""CREATE TABLE Resultado (
      id_equipo text,
      id_campeonato INTEGER,
      temporada text,
      posicion INTEGER,
      puntos INTEGER,
      FOREIGN KEY(id_equipo) REFERENCES Equipo(equipo_id),
      FOREIGN KEY(id_campeonato) REFERENCES Campeonato(campeonato_id)
    )""")

  db.commit

In [ ]:
def create_table_jugador():
  cursor = db.cursor()

  cursor.execute("""CREATE TABLE Jugador (
      jugador_id text,
      nombre text,
      nacionalidad text,
      peso integer,
      altura integer,
      fecha_nacimiento date,
      GK integer,
      DF integer,
      MF integer,
      FW integer,
      CB integer,
      FB integer,
      DM integer,
      CM integer,
      AM integer,
      WM integer
    )""")

  db.commit

In [ ]:
def format_name(column_name):
  return  column_name.replace(' - ','_minus_').replace('.','').replace('%','percentage').replace('+','_plus_').replace('/','_per_').replace(' (','_').replace(')','_').replace('(','_').replace(' ','_').replace('-','_')

In [ ]:
def create_CREATE_statement(table_name, columns):
  statement = "CREATE TABLE "+ table_name +" ( id_jugador text, id_campeonato INTEGER, temporada text, "
  for column in columns:
    column = format_name(column)
    statement = statement + column +' real, '

  statement = statement + 'FOREIGN KEY(id_jugador) REFERENCES JUGADOR(id_jugador), FOREIGN KEY(id_campeonato) REFERENCES Campeonato(campeonato_id) )'
  return statement

In [ ]:
def create_table_estadistica_jugador():
  table_name = "Estadisticas_jugador"
  columns = fields_jugadores
  
  statement = create_CREATE_statement(table_name, columns)

  cursor = db.cursor()

  cursor.execute(statement)
  
  db.commit()

In [ ]:
def create_table_estadistica_portero():
  table_name = "Estadisticas_portero"
  columns = fields_porteros
  
  statement = create_CREATE_statement(table_name, columns)

  cursor = db.cursor()

  cursor.execute(statement)
  
  db.commit()

In [ ]:
def create_table_contrato():
  cursor = db.cursor()

  cursor.execute("""CREATE TABLE Contrato (
      id_equipo text,
      id_jugador text,
      temporada text,
      matchs_played INTEGER,
      FOREIGN KEY(id_equipo) REFERENCES Equipo(equipo_id),
      FOREIGN KEY(id_jugador) REFERENCES Jugador(id_jugador)
    )""")

  db.commit

# C - Ejecución

In [ ]:
#MAIN NUEVO PREMIER LEAGUE

#Constantes
web_page = 'https://fbref.com/'
hasPercentile = True
#Variable global 
#Correr cada vez que ejecutemos el main
diccionario_posiciones = dict([
     ('GK', 6),
     ('DF', 7),
     ('MF', 8),
     ('FW', 9),
     ('CB', 10),
     ('FB', 11),
     ('DM', 12),
     ('CM', 13),
     ('AM', 14), 
     ('WM', 15),                             
  ])
diccionario_jugadores = {}
diccionario_equipos = {}
diccionario_ligas = dict([
     ('Premier League', 1),
     ('La Liga', 2),
     ('Bundesliga', 3),
     ('Serie A', 4),
     ('Ligue 1', 5),
     ('Major League Soccer', 6)
 ])

url_ligue = ['en/comps/9/Premier-League-Stats','Premier League']
url_jugador = 'https://fbref.com/en/players/2784f898/scout/365_euro/Eder-Militao-Scouting-Report'
url_portero = 'https://fbref.com/en/players/1840e36d/scout/365_euro/Thibaut-Courtois-Scouting-Report'

fields_jugadores = get_fields(url_jugador)
diccionario_fields_jugadores = dict.fromkeys(fields_jugadores)
fields_porteros = get_fields(url_portero)
diccionario_fields_porteros = dict.fromkeys(fields_porteros)

db = create_database("30032022-200858.db")
create_and_fill_table_campeonato()
create_table_equipo()
create_table_resultado()
create_table_jugador()
create_table_estadistica_jugador()
create_table_estadistica_portero()
create_table_contrato()

diccionario_equipos, diccionario_jugadores = get_ligue_all_years(url_ligue[0], url_ligue[1], diccionario_equipos, diccionario_jugadores) 

# create json object from dictionary
json_jugadores = json.dumps(diccionario_jugadores)
json_equipos = json.dumps(diccionario_equipos)
# open file for writing, "w" 
file_jugadores = open("diccionario_jugadores.json","w")
file_equipos = open("diccionario_equipos.json","w")
# write json object to file
file_jugadores.write(json_jugadores)
file_equipos.write(json_equipos)
# close file
file_jugadores.close()
file_equipos.close 

cursor = db.cursor()
cursor.execute("SELECT * FROM Resultado")
print(cursor.fetchall())
cursor.close()
db.close()

In [ ]:
#MAIN NUEVO LALIGA

#Constantes
web_page = 'https://fbref.com/'
hasPercentile = True
#Variable global 
#Correr cada vez que ejecutemos el main
diccionario_posiciones = dict([
     ('GK', 6),
     ('DF', 7),
     ('MF', 8),
     ('FW', 9),
     ('CB', 10),
     ('FB', 11),
     ('DM', 12),
     ('CM', 13),
     ('AM', 14), 
     ('WM', 15),                             
  ])

 
file_jugadores = open("diccionario_jugadores.json","r")
diccionario_jugadores = json.load(file_jugadores)
file_equipos = open("diccionario_equipos.json","r")
diccionario_equipos = json.load(file_equipos)
file_jugadores.close()
file_equipos.close()

diccionario_ligas = dict([
     ('Premier League', 1),
     ('La Liga', 2),
     ('Bundesliga', 3),
     ('Serie A', 4),
     ('Ligue 1', 5),
     ('Major League Soccer', 6)
 ])

url_ligue = ['en/comps/12/La-Liga-Stats','La Liga'] 
url_jugador = 'https://fbref.com/en/players/2784f898/scout/365_euro/Eder-Militao-Scouting-Report'
url_portero = 'https://fbref.com/en/players/1840e36d/scout/365_euro/Thibaut-Courtois-Scouting-Report'

fields_jugadores = get_fields(url_jugador)
diccionario_fields_jugadores = dict.fromkeys(fields_jugadores)
fields_porteros = get_fields(url_portero)
diccionario_fields_porteros = dict.fromkeys(fields_porteros)

all_ligue = []

db = sqlite3.connect("30032022-200858.db")

diccionario_equipos, diccionario_jugadores = get_ligue_all_years(url_ligue[0], url_ligue[1], diccionario_equipos, diccionario_jugadores) 

# create json object from dictionary
json_jugadores = json.dumps(diccionario_jugadores)
json_equipos = json.dumps(diccionario_equipos)
# open file for writing, "w" 
file_jugadores = open("diccionario_jugadores.json","w")
file_equipos = open("diccionario_equipos.json","w")
# write json object to file
file_jugadores.write(json_jugadores)
file_equipos.write(json_equipos)
# close file
file_jugadores.close()
file_equipos.close()

cursor = db.cursor()
cursor.execute("SELECT * FROM Resultado")
print(cursor.fetchall())
cursor.close()
db.close()

In [ ]:
#MAIN NUEVO BUNDESLIGA

#Constantes
web_page = 'https://fbref.com/'
hasPercentile = True
#Variable global 
#Correr cada vez que ejecutemos el main
diccionario_posiciones = dict([
     ('GK', 6),
     ('DF', 7),
     ('MF', 8),
     ('FW', 9),
     ('CB', 10),
     ('FB', 11),
     ('DM', 12),
     ('CM', 13),
     ('AM', 14), 
     ('WM', 15),                             
  ])

 
file_jugadores = open("diccionario_jugadores.json","r")
diccionario_jugadores = json.load(file_jugadores)
file_equipos = open("diccionario_equipos.json","r")
diccionario_equipos = json.load(file_equipos)
file_jugadores.close()
file_equipos.close()

diccionario_ligas = dict([
     ('Premier League', 1),
     ('La Liga', 2),
     ('Bundesliga', 3),
     ('Serie A', 4),
     ('Ligue 1', 5),
     ('Major League Soccer', 6)
 ])

url_ligue = ['en/comps/20/Bundesliga-Stats','Bundesliga']
url_jugador = 'https://fbref.com/en/players/2784f898/scout/365_euro/Eder-Militao-Scouting-Report'
url_portero = 'https://fbref.com/en/players/1840e36d/scout/365_euro/Thibaut-Courtois-Scouting-Report'

fields_jugadores = get_fields(url_jugador)
diccionario_fields_jugadores = dict.fromkeys(fields_jugadores)
fields_porteros = get_fields(url_portero)
diccionario_fields_porteros = dict.fromkeys(fields_porteros)

db = sqlite3.connect("30032022-200858.db")
  
diccionario_equipos, diccionario_jugadores = get_ligue_all_years(url_ligue[0], url_ligue[1], diccionario_equipos, diccionario_jugadores) 

# create json object from dictionary
json_jugadores = json.dumps(diccionario_jugadores)
json_equipos = json.dumps(diccionario_equipos)
# open file for writing, "w" 
file_jugadores = open("diccionario_jugadores.json","w")
file_equipos = open("diccionario_equipos.json","w")
# write json object to file
file_jugadores.write(json_jugadores)
file_equipos.write(json_equipos)
# close file
file_jugadores.close()
file_equipos.close()

cursor = db.cursor()
cursor.execute("SELECT * FROM Resultado")
print(cursor.fetchall())
cursor.close()
db.close()

In [ ]:
#MAIN NUEVO SERIE A

#Constantes
web_page = 'https://fbref.com/'
hasPercentile = True
#Variable global 
#Correr cada vez que ejecutemos el main
diccionario_posiciones = dict([
     ('GK', 6),
     ('DF', 7),
     ('MF', 8),
     ('FW', 9),
     ('CB', 10),
     ('FB', 11),
     ('DM', 12),
     ('CM', 13),
     ('AM', 14), 
     ('WM', 15),                             
  ])

 
file_jugadores = open("diccionario_jugadores.json","r")
diccionario_jugadores = json.load(file_jugadores)
file_equipos = open("diccionario_equipos.json","r")
diccionario_equipos = json.load(file_equipos)
file_jugadores.close()
file_equipos.close()

diccionario_ligas = dict([
     ('Premier League', 1),
     ('La Liga', 2),
     ('Bundesliga', 3),
     ('Serie A', 4),
     ('Ligue 1', 5),
     ('Major League Soccer', 6)
 ])

url_ligue = ['en/comps/11/Serie-A-Stats','Serie A']
url_jugador = 'https://fbref.com/en/players/2784f898/scout/365_euro/Eder-Militao-Scouting-Report'
url_portero = 'https://fbref.com/en/players/1840e36d/scout/365_euro/Thibaut-Courtois-Scouting-Report'

fields_jugadores = get_fields(url_jugador)
diccionario_fields_jugadores = dict.fromkeys(fields_jugadores)
fields_porteros = get_fields(url_portero)
diccionario_fields_porteros = dict.fromkeys(fields_porteros)

db = sqlite3.connect("30032022-200858.db")
  
diccionario_equipos, diccionario_jugadores = get_ligue_all_years(url_ligue[0], url_ligue[1], diccionario_equipos, diccionario_jugadores) 

# create json object from dictionary
json_jugadores = json.dumps(diccionario_jugadores)
json_equipos = json.dumps(diccionario_equipos)
# open file for writing, "w" 
file_jugadores = open("diccionario_jugadores.json","w")
file_equipos = open("diccionario_equipos.json","w")
# write json object to file
file_jugadores.write(json_jugadores)
file_equipos.write(json_equipos)
# close file
file_jugadores.close()
file_equipos.close()

cursor = db.cursor()
cursor.execute("SELECT * FROM Resultado")
print(cursor.fetchall())
cursor.close()
db.close()

In [ ]:

#MAIN NUEVO LIGUE 1

#Constantes
web_page = 'https://fbref.com/'
hasPercentile = True
#Variable global 
#Correr cada vez que ejecutemos el main
diccionario_posiciones = dict([
     ('GK', 6),
     ('DF', 7),
     ('MF', 8),
     ('FW', 9),
     ('CB', 10),
     ('FB', 11),
     ('DM', 12),
     ('CM', 13),
     ('AM', 14), 
     ('WM', 15),                             
  ])

 
file_jugadores = open("diccionario_jugadores.json","r")
diccionario_jugadores = json.load(file_jugadores)
file_equipos = open("diccionario_equipos.json","r")
diccionario_equipos = json.load(file_equipos)
file_jugadores.close()
file_equipos.close()

diccionario_ligas = dict([
     ('Premier League', 1),
     ('La Liga', 2),
     ('Bundesliga', 3),
     ('Serie A', 4),
     ('Ligue 1', 5),
     ('Major League Soccer', 6)
 ])

url_ligue = ['en/comps/13/Ligue-1-Stats','Ligue 1']
url_jugador = 'https://fbref.com/en/players/2784f898/scout/365_euro/Eder-Militao-Scouting-Report'
url_portero = 'https://fbref.com/en/players/1840e36d/scout/365_euro/Thibaut-Courtois-Scouting-Report'

fields_jugadores = get_fields(url_jugador)
diccionario_fields_jugadores = dict.fromkeys(fields_jugadores)
fields_porteros = get_fields(url_portero)
diccionario_fields_porteros = dict.fromkeys(fields_porteros)

db = sqlite3.connect("30032022-200858.db")
  
diccionario_equipos, diccionario_jugadores = get_ligue_all_years(url_ligue[0], url_ligue[1], diccionario_equipos, diccionario_jugadores) 

# create json object from dictionary
json_jugadores = json.dumps(diccionario_jugadores)
json_equipos = json.dumps(diccionario_equipos)
# open file for writing, "w" 
file_jugadores = open("diccionario_jugadores.json","w")
file_equipos = open("diccionario_equipos.json","w")
# write json object to file
file_jugadores.write(json_jugadores)
file_equipos.write(json_equipos)
# close file
file_jugadores.close()
file_equipos.close()

cursor = db.cursor()
cursor.execute("SELECT * FROM Resultado")
print(cursor.fetchall())
cursor.close()
db.close()